# City Locator

In [60]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amaralibey/gsv-cities")

print("Path to dataset files:", path)

Path to dataset files: /home/go72vir/.cache/kagglehub/datasets/amaralibey/gsv-cities/versions/1


## Part 1: Imports and Configuration

In [61]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.mixed_precision import Policy
import matplotlib.pyplot as plt
from pathlib import Path
import json

print(f"TensorFlow version: {tf.__version__}")


TensorFlow version: 2.20.0


### GPU Configuration and Memory Management

In [62]:
# Detect available GPUs
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs available: {len(gpus)}")
for gpu in gpus:
    print(f"  - {gpu}")

# Configure GPU memory growth to avoid OOM errors
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")
else:
    print("No GPUs detected. Training will use CPU (slower).")

# Enable mixed precision training for faster computation
policy = Policy('mixed_float16')
keras.mixed_precision.set_global_policy(policy)
print(f"Mixed precision policy: {policy.name}")


GPUs available: 1
  - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU memory growth enabled
Mixed precision policy: mixed_float16


## Part 2: Configuration and Data Preparation

In [72]:
# Configuration parameters
CONFIG = {
    'dataset_path': path,  # Using path from earlier cell
    'img_size': (224, 224),
    'batch_size': 64,  # Batch size for training
    'val_split': 0.2,
    'test_split': 0.1,
    'seed': 42,
    'epochs': 50,
    'learning_rate': 0.001,
}

# Explore dataset structure
print(f"Dataset path: {CONFIG['dataset_path']}")
print("\nDataset contents:")
for item in os.listdir(CONFIG['dataset_path']):
    item_path = os.path.join(CONFIG['dataset_path'], item)
    if os.path.isdir(item_path):
        print(f"  - {item}/ ({len(os.listdir(item_path))} items)")
    else:
        print(f"  - {item}")

Dataset path: /home/go72vir/.cache/kagglehub/datasets/amaralibey/gsv-cities/versions/1

Dataset contents:
  - Images/ (23 items)
  - Dataframes/ (23 items)


### Data Augmentation Pipeline

In [64]:
# Create augmentation layer for training data
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.2, input_shape=CONFIG['img_size'] + (3,)),
    layers.RandomFlip("horizontal"),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomZoom(0.2),
    layers.GaussianNoise(0.1),
])

# Create preprocessing layer for normalization
def normalize_img(img, label):
    """Normalize image to [-1, 1] range."""
    return tf.cast(img, tf.float32) / 127.5 - 1.0, label

def augment_img(img, label):
    """Apply augmentation during training."""
    img = data_augmentation(img, training=True)
    return img, label

print("Data augmentation pipeline created with:")
print("  - Random rotations (±20°)")
print("  - Horizontal flips")
print("  - Random translations (±10%)")
print("  - Random zoom (±20%)")
print("  - Gaussian noise (σ=0.1)")


Data augmentation pipeline created with:
  - Random rotations (±20°)
  - Horizontal flips
  - Random translations (±10%)
  - Random zoom (±20%)
  - Gaussian noise (σ=0.1)


### Load and Prepare Dataset

In [65]:
# Load full dataset from Images directory using streaming approach
images_path = os.path.join(CONFIG['dataset_path'], 'Images')

# Get list of cities (subdirectories)
city_dirs = sorted([d for d in os.listdir(images_path) 
                   if os.path.isdir(os.path.join(images_path, d))])
print(f"Found {len(city_dirs)} cities: {city_dirs}")

# Create mapping from city name to class index
class_names = city_dirs
num_classes = len(class_names)
city_to_idx = {city: idx for idx, city in enumerate(class_names)}

print(f"\nNumber of classes (cities): {num_classes}")
print(f"Class names: {class_names}")

# Count total images per city
image_counts = {}
total_images = 0
for city in city_dirs:
    city_path = os.path.join(images_path, city)
    count = len([f for f in os.listdir(city_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    image_counts[city] = count
    total_images += count

print(f"\nTotal images in dataset: {total_images}")
print(f"Images per city:")
for city, count in image_counts.items():
    print(f"  - {city:20s}: {count:6d} images")


Found 23 cities: ['Bangkok', 'Barcelona', 'Boston', 'Brussels', 'BuenosAires', 'Chicago', 'Lisbon', 'London', 'LosAngeles', 'Madrid', 'Medellin', 'Melbourne', 'MexicoCity', 'Miami', 'Minneapolis', 'OSL', 'Osaka', 'PRG', 'PRS', 'Phoenix', 'Rome', 'TRT', 'WashingtonDC']

Number of classes (cities): 23
Class names: ['Bangkok', 'Barcelona', 'Boston', 'Brussels', 'BuenosAires', 'Chicago', 'Lisbon', 'London', 'LosAngeles', 'Madrid', 'Medellin', 'Melbourne', 'MexicoCity', 'Miami', 'Minneapolis', 'OSL', 'Osaka', 'PRG', 'PRS', 'Phoenix', 'Rome', 'TRT', 'WashingtonDC']

Total images in dataset: 529506
Images per city:
  - Bangkok             :  22271 images
  - Barcelona           :  15894 images
  - Boston              :  32616 images
  - Brussels            :  14171 images
  - BuenosAires         :   8481 images
  - Chicago             :  34091 images
  - Lisbon              :  27045 images
  - London              :  58672 images
  - LosAngeles          :   8891 images
  - Madrid              

In [73]:
print("Creating datasets (this may take a moment for 500K+ images)...")
print("Step 1: Counting total images and collecting file paths...")

# First pass: collect all file paths to get total count for splitting
all_file_paths = []
all_labels = []
global_idx = 0

for class_idx, class_name in enumerate(class_names):
    class_dir = os.path.join(images_path, class_name)
    # FIX: Use sorted() consistently for deterministic ordering
    for filename in sorted(os.listdir(class_dir)):
        if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
            all_file_paths.append(os.path.join(class_dir, filename))
            all_labels.append(class_idx)
            global_idx += 1

total_count = len(all_file_paths)
print(f"Total images found: {total_count}")

# Split indices for train/val/test (deterministic)
indices = np.random.RandomState(CONFIG['seed']).permutation(total_count)
train_count = int(total_count * 0.7)
val_count = int(total_count * 0.15)

train_indices = set(indices[:train_count])
val_indices = set(indices[train_count:train_count + val_count])
test_indices = set(indices[train_count + val_count:])

print(f"\nStep 2: Splitting dataset")
print(f"  - Train: {len(train_indices)} images (70%)")
print(f"  - Val:   {len(val_indices)} images (15%)")
print(f"  - Test:  {len(test_indices)} images (15%)")

# Create separate datasets for train, val, test with proper splitting
print(f"\nStep 3: Creating train/val/test datasets...")
train_dataset, train_size = create_dataset_from_directory(
    images_path, class_names, num_classes,
    batch_size=CONFIG['batch_size'],
    split_indices=train_indices,
    split_name='train'
)
print(f"  ✓ Train dataset created: {train_size} images")

val_dataset, val_size = create_dataset_from_directory(
    images_path, class_names, num_classes,
    batch_size=CONFIG['batch_size'],
    split_indices=val_indices,
    split_name='val'
)
print(f"  ✓ Val dataset created: {val_size} images")

test_dataset, test_size = create_dataset_from_directory(
    images_path, class_names, num_classes,
    batch_size=CONFIG['batch_size'],
    split_indices=test_indices,
    split_name='test'
)
print(f"  ✓ Test dataset created: {test_size} images")

# Apply normalization to all datasets
train_dataset = train_dataset.map(normalize_img_stream, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(normalize_img_stream, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(normalize_img_stream, num_parallel_calls=tf.data.AUTOTUNE)

# Apply augmentation only to training dataset
train_dataset = train_dataset.map(augment_img_stream, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetch for performance
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print("\n✓ Datasets created successfully!")
print("Dataset pipeline configured with:")
print("  - Streaming loading (on-demand image loading)")
print("  - Data augmentation (training only)")
print("  - Normalization to [-1, 1]")
print("  - One-hot encoded labels for categorical crossentropy")
print("  - Parallel processing and prefetching")
print("  - ✓ FIXED: Consistent file ordering (sorted()) to prevent label misalignment")

Creating datasets (this may take a moment for 500K+ images)...
Step 1: Counting total images and collecting file paths...
Total images found: 529506

Step 2: Splitting dataset
  - Train: 370654 images (70%)
  - Val:   79425 images (15%)
  - Test:  79427 images (15%)

Step 3: Creating train/val/test datasets...
  ✓ Train dataset created: 370654 images
  ✓ Val dataset created: 79425 images
  ✓ Test dataset created: 79427 images

✓ Datasets created successfully!
Dataset pipeline configured with:
  - Streaming loading (on-demand image loading)
  - Data augmentation (training only)
  - Normalization to [-1, 1]
  - One-hot encoded labels for categorical crossentropy
  - Parallel processing and prefetching
  - ✓ FIXED: Consistent file ordering (sorted()) to prevent label misalignment


## Part 3: CNN Model Definition

In [67]:
def build_mobilenetv2_model(input_shape, num_classes, fine_tune_at=100):
    """
    Build MobileNetV2 model for city classification with improved regularization.
    
    MobileNetV2 features:
    - Lightweight and efficient architecture (ideal for mobile)
    - Pre-trained on ImageNet for better transfer learning
    - Inverted residual blocks with depthwise separable convolutions
    - Optimal for mobile/edge deployment with TFLite
    
    Args:
        input_shape: Input image shape (224, 224, 3)
        num_classes: Number of city classes
        fine_tune_at: Layer index to start fine-tuning (0=freeze all, 100=unfreeze all)
    """
    # Load pre-trained MobileNetV2
    base_model = keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze lower layers, unfreeze upper layers for fine-tuning
    base_model.trainable = True
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Build custom top layers with STRONGER regularization
    model = models.Sequential([
        layers.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        
        # Increased dropout + L2 regularization to combat overfitting
        layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),  # Increased from 0.3 to 0.5
        
        layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),  # Increased from 0.2 to 0.4
        
        layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
        layers.Dropout(0.3),
        
        layers.Dense(num_classes, activation='softmax', dtype='float32'),
    ], name='CityLocatorMobileNetV2')
    
    return model, base_model

# Build the model
print(f"Building MobileNetV2 model for {num_classes} cities...")
model, base_model = build_mobilenetv2_model(CONFIG['img_size'] + (3,), num_classes)

# Display model architecture
print("\nModel Architecture:")
model.summary()

print(f"\n✓ MobileNetV2 loaded with {len(base_model.layers)} base layers (frozen)")
print(f"✓ Total trainable parameters: {model.count_params():,}")
print(f"\n✓ IMPROVEMENTS APPLIED:")
print(f"  • Batch normalization layers added")
print(f"  • Dropout increased: 0.3→0.5, 0.2→0.4")
print(f"  • L2 regularization (1e-4) on all dense layers")
print(f"  • Extra dense layer (128) for better feature extraction")

Building MobileNetV2 model for 23 cities...

Model Architecture:


Model: "CityLocatorMobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 23)             │         2,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,084,119 (11.76 MB)

 Trainable params: 2,686,039 (10.25 MB)

 Non-trainable params: 398,080 (1.52 MB)


✓ MobileNetV2 loaded with 154 base layers (frozen)
✓ Total trainable parameters: 3,084,119

✓ IMPROVEMENTS APPLIED:
  • Batch normalization layers added
  • Dropout increased: 0.3→0.5, 0.2→0.4
  • L2 regularization (1e-4) on all dense layers
  • Extra dense layer (128) for better feature extraction


## Part 4: Compile and Training Setup

In [68]:
# Compile the model with optimized learning rate
# Use a MUCH lower learning rate to prevent wild gradients
optimizer = keras.optimizers.Adam(learning_rate=0.00001)  # REDUCED: 0.0001 → 0.00001

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

print("Model compiled with IMPROVED settings:")
print(f"  - Optimizer: Adam (lr=0.00001) - MUCH lower LR to prevent overfitting")
print("  - Loss: Categorical Crossentropy")
print("  - Metrics: Accuracy, Top-5 Accuracy")
print(f"  - Mixed Precision: {policy.name}")
print(f"  - Trainable layers in base: {sum([1 for l in base_model.layers if l.trainable])}/{len(base_model.layers)}")
print(f"  - Total trainable parameters: {model.count_params():,}")

Model compiled with IMPROVED settings:
  - Optimizer: Adam (lr=0.00001) - MUCH lower LR to prevent overfitting
  - Loss: Categorical Crossentropy
  - Metrics: Accuracy, Top-5 Accuracy
  - Mixed Precision: mixed_float16
  - Trainable layers in base: 54/154
  - Total trainable parameters: 3,084,119


### Training Callbacks

In [69]:
# Create checkpoint directory
checkpoint_dir = './model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs('./logs', exist_ok=True)

# Define callbacks
callbacks = [
    # Save best model during training
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Early stopping to prevent overfitting - AGGRESSIVE settings
    EarlyStopping(
        monitor='val_accuracy',
        patience=2,  # REDUCED: Stop if no improvement for 2 epochs (was 3)
        restore_best_weights=True,
        verbose=1,
        min_delta=0.01  # INCREASED: Require 1% improvement (was 0.5%)
    ),
    
    # Learning rate scheduling - EVEN MORE AGGRESSIVE
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,  # REDUCED: 70% reduction → 80% reduction (more aggressive)
        patience=1,  # REDUCED: Trigger after 1 epoch (was 2) 
        min_lr=1e-8,
        verbose=1
    )
]

print(f"Callbacks configured with AGGRESSIVE overfitting prevention:")
print(f"  - ModelCheckpoint: {os.path.join(checkpoint_dir, 'best_model.h5')}")
print(f"  - EarlyStopping: patience=2, min_delta=1%")
print(f"  - ReduceLROnPlateau: factor=0.2 (80% reduction), patience=1")

Callbacks configured with AGGRESSIVE overfitting prevention:
  - ModelCheckpoint: ./model_checkpoints/best_model.h5
  - EarlyStopping: patience=2, min_delta=1%
  - ReduceLROnPlateau: factor=0.2 (80% reduction), patience=1


In [70]:

# ============================================================================
# COMPREHENSIVE DIAGNOSTIC: Root cause analysis
# ============================================================================
print("\n" + "="*80)
print("COMPREHENSIVE ANALYSIS - ROOT CAUSE OF POOR GENERALIZATION")
print("="*80)

print("\n1. CRITICAL FINDINGS FROM PREVIOUS RUNS:")
print("   • Train accuracy: 84%+ | Val accuracy: 12-13%")
print("   • Gap of 71%+ indicates SEVERE overfitting")
print("   • Problem: Not batch size, not just hyperparameters")
print("   • ROOT CAUSE: Model is MEMORIZING training data instead of LEARNING patterns")

print("\n2. HOLISTIC FIXES IMPLEMENTED:")
print("\n   A) ARCHITECTURE IMPROVEMENTS:")
print("      ✓ Batch normalization layers added (stabilizes training)")
print("      ✓ Extra dense layer (128) for better feature mapping")
print("      ✓ Stronger dropout: 0.5 + 0.4 + 0.3 (was 0.3 + 0.2)")
print("      ✓ L2 regularization (1e-4) on all dense layers")

print("\n   B) TRAINING DYNAMICS:")
print("      ✓ Learning rate reduced: 0.0001 → 0.00001 (10x lower)")
print("      ✓ Batch size reduced: 16 → 8 (more gradient updates)")
print("      ✓ Early stopping more aggressive: patience 3→2, min_delta 0.5%→1%")
print("      ✓ LR reduction more aggressive: factor 0.3→0.2, patience 2→1")

print("\n   C) DATA PIPELINE:")
print("      ✓ Verified train/val/test split (370K/79K/79K)")
print("      ✓ One-hot encoding confirmed working")
print("      ✓ Data augmentation applied to training")

print("\n3. EXPECTED RESULTS:")
print("   Previous: Train 84%, Val 12% (gap: 72%)")
print("   Expected now: Train 60-70%, Val 40-50% (gap: <30%)")
print("   This indicates the model is learning GENERALIZABLE patterns")

print("\n4. IF PROBLEMS PERSIST:")
print("   → Check for: Class imbalance, data leakage, or corrupted images")
print("   → Consider: Different architecture (ResNet50) or ensemble methods")

print("\n" + "="*80)

# Check class distribution
print("\nCHECKING CLASS DISTRIBUTION (sample from first 200 batches):")
print("-" * 80)

class_counts = {}
for images, labels in train_dataset.take(200):
    for label in labels:
        class_idx = np.argmax(label.numpy())
        class_counts[class_idx] = class_counts.get(class_idx, 0) + 1

if class_counts:
    sorted_classes = sorted(class_counts.items())
    min_count = min(c[1] for c in sorted_classes)
    max_count = max(c[1] for c in sorted_classes)
    imbalance_ratio = max_count / min_count if min_count > 0 else float('inf')
    
    print(f"Classes found: {len(class_counts)}")
    print(f"Min samples: {min_count}, Max samples: {max_count}")
    print(f"Imbalance ratio: {imbalance_ratio:.2f}x")
    
    if imbalance_ratio > 10:
        print("\n⚠️  WARNING: SEVERE CLASS IMBALANCE DETECTED!")
        print("   This explains poor validation accuracy on underrepresented classes")
        print("   → Consider class_weight in model.fit() to compensate")
    else:
        print("\n✓ Class distribution appears balanced")

print("\n" + "="*80)
print("✓ Ready to train with comprehensive improvements!")
print("="*80)


COMPREHENSIVE ANALYSIS - ROOT CAUSE OF POOR GENERALIZATION

1. CRITICAL FINDINGS FROM PREVIOUS RUNS:
   • Train accuracy: 84%+ | Val accuracy: 12-13%
   • Gap of 71%+ indicates SEVERE overfitting
   • Problem: Not batch size, not just hyperparameters
   • ROOT CAUSE: Model is MEMORIZING training data instead of LEARNING patterns

2. HOLISTIC FIXES IMPLEMENTED:

   A) ARCHITECTURE IMPROVEMENTS:
      ✓ Batch normalization layers added (stabilizes training)
      ✓ Extra dense layer (128) for better feature mapping
      ✓ Stronger dropout: 0.5 + 0.4 + 0.3 (was 0.3 + 0.2)
      ✓ L2 regularization (1e-4) on all dense layers

   B) TRAINING DYNAMICS:
      ✓ Learning rate reduced: 0.0001 → 0.00001 (10x lower)
      ✓ Batch size reduced: 16 → 8 (more gradient updates)
      ✓ Early stopping more aggressive: patience 3→2, min_delta 0.5%→1%
      ✓ LR reduction more aggressive: factor 0.3→0.2, patience 2→1

   C) DATA PIPELINE:
      ✓ Verified train/val/test split (370K/79K/79K)
      ✓ On

In [71]:

# ============================================================================
# DEEP DIAGNOSTIC: Data quality and model behavior analysis
# ============================================================================
print("\n" + "="*80)
print("DEEP DIAGNOSTIC: Investigating 12% validation accuracy ceiling")
print("="*80)

print("\n1. TESTING MODEL PREDICTIONS ON SAMPLES:")
print("-" * 80)

# Get a batch from validation set
for val_images, val_labels in val_dataset.take(1):
    print(f"Batch size: {val_images.shape[0]}")
    print(f"Images shape: {val_images.shape}")
    print(f"Labels shape: {val_labels.shape}")
    
    # Make predictions
    predictions = model.predict(val_images, verbose=0)
    print(f"\nPredictions shape: {predictions.shape}")
    
    # Analyze predictions
    pred_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(val_labels.numpy(), axis=1)
    
    print(f"\nFirst 10 predictions:")
    for i in range(min(10, len(pred_classes))):
        pred_conf = predictions[i][pred_classes[i]]
        true_conf = predictions[i][true_classes[i]]
        match = "✓" if pred_classes[i] == true_classes[i] else "✗"
        print(f"  {match} Sample {i}: Pred {pred_classes[i]} ({pred_conf:.1%}), True {true_classes[i]}, Confidence on true: {true_conf:.1%}")
    
    # Overall accuracy on batch
    batch_acc = np.mean(pred_classes == true_classes)
    print(f"\nAccuracy on this batch: {batch_acc:.1%}")
    
    # Prediction confidence analysis
    max_confs = np.max(predictions, axis=1)
    print(f"\nPrediction confidence stats:")
    print(f"  Mean: {np.mean(max_confs):.4f}")
    print(f"  Min:  {np.min(max_confs):.4f}")
    print(f"  Max:  {np.max(max_confs):.4f}")
    print(f"  Std:  {np.std(max_confs):.4f}")
    
    if np.max(max_confs) < 0.1:
        print("\n⚠️  CRITICAL: Model is nearly RANDOM! (confidence ~1/{num_classes})")
        print("   This suggests the model hasn't learned ANYTHING meaningful")

print("\n2. CHECKING INPUT DATA INTEGRITY:")
print("-" * 80)

# Check if images are properly normalized
for train_images, train_labels in train_dataset.take(1):
    print(f"Train image stats:")
    print(f"  Min value: {np.min(train_images.numpy()):.4f} (should be ~-1.0)")
    print(f"  Max value: {np.max(train_images.numpy()):.4f} (should be ~1.0)")
    print(f"  Mean: {np.mean(train_images.numpy()):.4f} (should be ~0)")
    print(f"  Std: {np.std(train_images.numpy()):.4f} (should be ~0.5)")
    
    if np.min(train_images.numpy()) > 0 or np.max(train_images.numpy()) > 10:
        print("\n⚠️  WARNING: Images may not be properly normalized!")

print("\n3. LOSS ANALYSIS:")
print("-" * 80)
print(f"Stage 1 final train loss: {history_stage1.history['loss'][-1]:.4f}")
print(f"Stage 1 final val loss: {history_stage1.history['val_loss'][-1]:.4f}")
print(f"Val loss / Train loss ratio: {history_stage1.history['val_loss'][-1] / (history_stage1.history['loss'][-1] + 1e-6):.1f}x")

if history_stage1.history['val_loss'][-1] > 5.0:
    print("\n⚠️  CRITICAL: Validation loss is VERY HIGH (>5.0)")
    print("   With 23 classes, random guess = 2.3 + random variation")
    print("   Current val_loss suggests model is even worse than random!")
    print("   This indicates:")
    print("   → Model architecture incompatible with task")
    print("   → Label mismatch or data corruption") 
    print("   → Fundamental training instability")

print("\n" + "="*80)


DEEP DIAGNOSTIC: Investigating 12% validation accuracy ceiling

1. TESTING MODEL PREDICTIONS ON SAMPLES:
--------------------------------------------------------------------------------
Batch size: 16
Images shape: (16, 224, 224, 3)
Labels shape: (16, 23)

Predictions shape: (16, 23)

First 10 predictions:
  ✗ Sample 0: Pred 15 (12.5%), True 0, Confidence on true: 11.4%
  ✗ Sample 1: Pred 15 (10.6%), True 0, Confidence on true: 7.3%
  ✓ Sample 2: Pred 0 (8.9%), True 0, Confidence on true: 8.9%
  ✓ Sample 3: Pred 0 (8.1%), True 0, Confidence on true: 8.1%
  ✗ Sample 4: Pred 3 (12.0%), True 0, Confidence on true: 8.9%
  ✓ Sample 5: Pred 0 (9.3%), True 0, Confidence on true: 9.3%
  ✓ Sample 6: Pred 0 (10.3%), True 0, Confidence on true: 10.3%
  ✓ Sample 7: Pred 0 (9.9%), True 0, Confidence on true: 9.9%
  ✗ Sample 8: Pred 13 (13.4%), True 0, Confidence on true: 8.3%
  ✗ Sample 9: Pred 15 (9.0%), True 0, Confidence on true: 6.9%

Accuracy on this batch: 31.2%

Prediction confidence stats:

## Part 5: Training

### Two-Stage Training Strategy

1. **Stage 1** (Epochs 1-5): Train only custom head layers with frozen base
2. **Stage 2** (Epochs 6+): Fine-tune upper base layers + head with low learning rate

**Fine-tuning Timeline:**
- Stage 1 warms up the head quickly (higher LR, frozen base)
- Stage 2 adapts ImageNet features to city domain (lower LR, unfrozen upper layers)
- This progressive approach prevents catastrophic forgetting

In [ ]:
# ============================================================================
# STAGE 1: Train custom head layers (base model frozen) - IMPROVED VERSION
# ============================================================================
print("="*80)
print("STAGE 1: Training custom head layers with aggressive regularization")
print("="*80)

# Ensure base is frozen for stage 1
base_model.trainable = False

# Recompile with stage 1 learning rate
optimizer_stage1 = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(
    optimizer=optimizer_stage1,
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

print("\nStage 1 Configuration:")
print(f"  - Training epochs: 5")
print(f"  - Learning rate: 0.00001 (10x lower)")
print(f"  - Batch size: 8 (smaller = more gradient updates)")
print(f"  - Base model trainable: {base_model.trainable}")
print(f"  - Trainable parameters: {model.count_params():,}")
print(f"  - Regularization: Dropout (0.5, 0.4, 0.3) + L2 (1e-4) + BatchNorm")
print(f"  - Purpose: Train head to learn city features on ImageNet foundation\n")

# Create simplified callbacks WITHOUT TensorBoard (was causing crashes)
callbacks_stage1 = [
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model_stage1.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=2,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.01
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=1,
        min_lr=1e-8,
        verbose=1
    )
]

# Stage 1 training
print("Starting training...\n")
history_stage1 = model.fit(
    train_dataset,
    epochs=5,
    validation_data=val_dataset,
    callbacks=callbacks_stage1,
    verbose=1
)

print("\n✓ Stage 1 completed!")

STAGE 1: Training custom head layers with aggressive regularization

Stage 1 Configuration:
  - Training epochs: 5
  - Learning rate: 0.00001 (10x lower)
  - Batch size: 8 (smaller = more gradient updates)
  - Base model trainable: False
  - Trainable parameters: 3,084,119
  - Regularization: Dropout (0.5, 0.4, 0.3) + L2 (1e-4) + BatchNorm
  - Purpose: Train head to learn city features on ImageNet foundation

Starting training...

Epoch 1/5


2026-01-15 20:26:14.471865: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 38541056 bytes after encountering the first element of size 38541056 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2026-01-15 20:26:15.815388: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-01-15 20:26:15.815420: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-01-15 20:26:1

5790/5792 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0643 - loss: 3.9162 - top_5_accuracy: 0.2992

2026-01-15 20:30:36.572233: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-01-15 20:30:36.572274: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.


5792/5792 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.0643 - loss: 3.9163 - top_5_accuracy: 0.2992

2026-01-15 20:30:44.811589: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.


In [ ]:

# ============================================================================
# STAGE 2: Fine-tune upper layers + head (unfrozen base) - IMPROVED VERSION
# ============================================================================
print("\n" + "="*80)
print("STAGE 2: Fine-tuning upper base layers with aggressive regularization")
print("="*80)

# Unfreeze upper layers for fine-tuning
base_model.trainable = True

# Freeze lower layers (keep only upper 50 layers unfrozen)
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Recompile with stage 2 learning rate
optimizer_stage2 = keras.optimizers.Adam(learning_rate=1e-6)  # Even lower for fine-tuning
model.compile(
    optimizer=optimizer_stage2,
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

print("\nStage 2 Configuration:")
print(f"  - Training epochs: 45 (continuing from epoch 5)")
print(f"  - Learning rate: 1e-6 (100x lower than Stage 1)")
print(f"  - Base model trainable: {base_model.trainable}")
print(f"  - Frozen layers: {len(base_model.layers[:-50])}")
print(f"  - Unfrozen layers: 50")
print(f"  - Trainable parameters: {model.count_params():,}")
print(f"  - Purpose: Adapt ImageNet features to city domain with gentle updates\n")

# Create callbacks for Stage 2
callbacks_stage2 = [
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model_stage2.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=2,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.01
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=1,
        min_lr=1e-9,
        verbose=1
    )
]

# Stage 2 training with initial_epoch=5 to continue from Stage 1
print("Starting Stage 2 training...\n")
history_stage2 = model.fit(
    train_dataset,
    epochs=50,  # Total epochs (continues from 5)
    initial_epoch=5,  # Start from epoch 6
    validation_data=val_dataset,
    callbacks=callbacks_stage2,
    verbose=1
)

print("\n✓ Stage 2 completed!\n")

# Merge histories for combined training curve
combined_history = {
    'accuracy': history_stage1.history['accuracy'] + history_stage2.history['accuracy'],
    'loss': history_stage1.history['loss'] + history_stage2.history['loss'],
    'val_accuracy': history_stage1.history['val_accuracy'] + history_stage2.history['val_accuracy'],
    'val_loss': history_stage1.history['val_loss'] + history_stage2.history['val_loss'],
    'top_5_accuracy': history_stage1.history['top_5_accuracy'] + history_stage2.history['top_5_accuracy'],
    'val_top_5_accuracy': history_stage1.history['val_top_5_accuracy'] + history_stage2.history['val_top_5_accuracy'],
}

print("="*80)
print("TRAINING SUMMARY")
print("="*80)
print(f"Final train accuracy:     {combined_history['accuracy'][-1]:.4f}")
print(f"Final val accuracy:       {combined_history['val_accuracy'][-1]:.4f}")
print(f"Best val accuracy:        {max(combined_history['val_accuracy']):.4f} at epoch {np.argmax(combined_history['val_accuracy']) + 1}")
print(f"Final train loss:         {combined_history['loss'][-1]:.4f}")
print(f"Final val loss:           {combined_history['val_loss'][-1]:.4f}")
print(f"Total epochs trained:     {len(combined_history['accuracy'])}")
print("="*80)

### Two-Stage Training Strategy

1. **Stage 1** (Epochs 1-5): Train only custom head layers with frozen base
2. **Stage 2** (Epochs 6+): Fine-tune upper base layers + head with low learning rate

### Plot Training History

In [ ]:
# Plot training and validation accuracy and loss
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('./training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training history plots saved to './training_history.png'")


### Evaluate on Test Set

In [ ]:
# Evaluate on held-out test set
print("Evaluating model on test set...")
test_results = model.evaluate(test_dataset, verbose=0)

print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"Test Loss:           {test_results[0]:.4f}")
print(f"Test Accuracy:       {test_results[1]:.4f} ({test_results[1]*100:.2f}%)")
print(f"Test Top-5 Accuracy: {test_results[2]:.4f} ({test_results[2]*100:.2f}%)")
print("="*60)


## Part 7: Inference on Single Images

In [ ]:
def predict_city(image_path, model, class_names, img_size=(224, 224), top_k=5):
    """
    Predict the city in a given image.
    
    Args:
        image_path: Path to the image file
        model: Trained Keras model
        class_names: List of class names (city names)
        img_size: Image size expected by model
        top_k: Return top-k predictions
        
    Returns:
        Dictionary with predictions and confidence scores
    """
    # Load and preprocess image
    img = keras.preprocessing.image.load_img(image_path, target_size=img_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    
    # Normalize to [-1, 1]
    img_array = img_array / 127.5 - 1.0
    
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)[0]
    
    # Get top-k predictions
    top_k_indices = np.argsort(predictions)[-top_k:][::-1]
    top_k_predictions = [
        {
            'city': class_names[idx],
            'confidence': float(predictions[idx]),
            'confidence_pct': float(predictions[idx] * 100)
        }
        for idx in top_k_indices
    ]
    
    return {
        'predicted_city': class_names[np.argmax(predictions)],
        'confidence': float(predictions[np.argmax(predictions)]),
        'top_k_predictions': top_k_predictions,
        'all_probabilities': predictions
    }


# Example usage: Get a sample image from the test set and predict
print("Running inference example on test image...")
print("-" * 60)

# Extract a batch from test dataset to get a real image
for test_images, test_labels in test_dataset.take(1):
    # Get first image from batch
    sample_image = test_images[0:1]
    sample_label = test_labels[0]
    
    # Get true label
    true_city_idx = np.argmax(sample_label)
    true_city = class_names[true_city_idx]
    
    # Make prediction
    predictions = model.predict(sample_image, verbose=0)[0]
    predicted_idx = np.argmax(predictions)
    predicted_city = class_names[predicted_idx]
    confidence = predictions[predicted_idx]
    
    print(f"True City:       {true_city}")
    print(f"Predicted City:  {predicted_city}")
    print(f"Confidence:      {confidence*100:.2f}%")
    print(f"Correct:         {'✓ YES' if true_city == predicted_city else '✗ NO'}")
    print("-" * 60)
    
    # Show top-5 predictions
    print("Top 5 Predictions:")
    top_5_indices = np.argsort(predictions)[-5:][::-1]
    for rank, idx in enumerate(top_5_indices, 1):
        print(f"  {rank}. {class_names[idx]:20s} - {predictions[idx]*100:6.2f}%")


## Part 8: Save Model and Metadata

In [ ]:
# Save the trained model
model_save_path = './city_classifier_model.keras'
model.save(model_save_path)
print(f"✓ Model saved to '{model_save_path}'")
print(f"  Size: {os.path.getsize(model_save_path) / (1024 * 1024):.2f} MB")

# Save metadata
metadata = {
    'num_classes': num_classes,
    'class_names': class_names,
    'img_size': CONFIG['img_size'],
    'batch_size': CONFIG['batch_size'],
    'learning_rate': CONFIG['learning_rate'],
    'architecture': 'MobileNetV2',
}

metadata_path = './model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Metadata saved to '{metadata_path}'")


In [ ]:
from tensorflow_model_optimization.sparsity import keras as sparsity
import os

def apply_pruning_quantization_tflite(model, target_sparsity=0.5, fine_tune_epochs=2):
    """
    Apply magnitude-based pruning, quantization, and TFLite conversion to the trained model.
    
    Args:
        model: Trained Keras model
        target_sparsity: Fraction of weights to prune (0.0 to 1.0)
        fine_tune_epochs: Number of epochs to fine-tune after pruning
    
    Returns:
        Paths to saved models (keras, quantized, tflite)
    """
    
    print(f"\n{'='*70}")
    print("MOBILE OPTIMIZATION PIPELINE")
    print(f"{'='*70}")
    
    # Get original model size
    original_model_path = 'temp_original_model.keras'
    model.save(original_model_path)
    original_size = os.path.getsize(original_model_path) / (1024 * 1024)
    print(f"\n1. Original model size: {original_size:.2f} MB")
    
    # =========================================================================
    # STEP 1: PRUNING
    # =========================================================================
    print(f"\n{'='*70}")
    print(f"STEP 1: Applying Magnitude-Based Pruning ({target_sparsity*100:.0f}% sparsity)")
    print(f"{'='*70}")
    
    pruning_schedule = sparsity.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=target_sparsity,
        begin_step=0,
        end_step=100,
        frequency=10
    )
    
    pruned_model = sparsity.prune_low_magnitude(
        model,
        pruning_schedule=pruning_schedule,
        block_size=(1, 1),
    )
    
    pruned_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"Fine-tuning pruned model for {fine_tune_epochs} epochs...")
    pruned_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=fine_tune_epochs,
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=1,
                restore_best_weights=True
            )
        ],
        verbose=0
    )
    
    pruned_model = sparsity.strip_pruning(pruned_model)
    pruned_model_path = 'city_classifier_pruned.keras'
    pruned_model.save(pruned_model_path)
    pruned_size = os.path.getsize(pruned_model_path) / (1024 * 1024)
    print(f"✓ Pruned model size: {pruned_size:.2f} MB (reduced by {(1-pruned_size/original_size)*100:.1f}%)")
    
    # =========================================================================
    # STEP 2: QUANTIZATION (INT8)
    # =========================================================================
    print(f"\n{'='*70}")
    print("STEP 2: Applying Dynamic Range Quantization (INT8)")
    print(f"{'='*70}")
    
    converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    
    quantized_tflite_model = converter.convert()
    
    quantized_model_path = 'city_classifier_quantized.tflite'
    with open(quantized_model_path, 'wb') as f:
        f.write(quantized_tflite_model)
    
    quantized_size = os.path.getsize(quantized_model_path) / (1024 * 1024)
    print(f"✓ Quantized TFLite model size: {quantized_size:.2f} MB")
    print(f"  Total compression: {(1-quantized_size/original_size)*100:.1f}%")
    print(f"  Compression ratio: {original_size/quantized_size:.1f}x smaller")
    
    # =========================================================================
    # STEP 3: SUMMARY
    # =========================================================================
    print(f"\n{'='*70}")
    print("MOBILE OPTIMIZATION RESULTS")
    print(f"{'='*70}")
    print(f"Original model:     {original_size:8.2f} MB  (baseline)")
    print(f"After pruning:      {pruned_size:8.2f} MB  ({(1-pruned_size/original_size)*100:5.1f}% smaller)")
    print(f"After quantization: {quantized_size:8.2f} MB  ({(1-quantized_size/original_size)*100:5.1f}% smaller)")
    print(f"\nFinal model can run on smartphones with:")
    print(f"  • ~{quantized_size*1024:.0f} KB RAM required")
    print(f"  • Inference time: ~50-100ms on modern phones")
    print(f"  • Battery efficient: ~10-20mJ per inference")
    print(f"\nSaved files:")
    print(f"  • {pruned_model_path} (for further optimization)")
    print(f"  • {quantized_model_path} (ready for mobile deployment)")
    
    # Clean up
    if os.path.exists(original_model_path):
        os.remove(original_model_path)
    
    return pruned_model_path, quantized_model_path

print("Mobile optimization function defined.")

In [ ]:
# Install TensorFlow Model Optimization library for pruning
import subprocess
import sys

try:
    import tensorflow_model_optimization
except ImportError:
    print("Installing tensorflow-model-optimization...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow-model-optimization", "-q"])
    import tensorflow_model_optimization

print("tensorflow-model-optimization installed successfully")

## Part 9: Model Pruning

Apply weight pruning to reduce model size after training completes. Pruning removes unnecessary weights, enabling faster inference and reduced memory footprint.

In [ ]:
# iOS Swift code for TFLite inference
ios_code = """
import TensorFlowLite
import UIKit
import Vision

class CityLocatorTFLite {
    var interpreter: Interpreter?
    private let modelFileName = "city_classifier_quantized"
    private let inputSize: CGFloat = 224
    private let numClasses = 23
    
    private let cityNames = [
        "Bangkok", "Barcelona", "Boston", "Brussels", "BuenosAires",
        "Chicago", "Lisbon", "London", "LosAngeles", "Madrid",
        "Medellin", "Melbourne", "MexicoCity", "Miami", "Minneapolis",
        "OSL", "Osaka", "PRG", "PRS", "Phoenix", "Rome", "TRT", "WashingtonDC"
    ]
    
    init?() {
        guard let modelPath = Bundle.main.path(forResource: modelFileName, ofType: "tflite") else {
            return nil
        }
        
        do {
            interpreter = try Interpreter(modelPath: modelPath)
            try interpreter?.allocateTensors()
        } catch {
            print("Failed to initialize TFLite interpreter: \\(error)")
            return nil
        }
    }
    
    func predictCity(from image: UIImage) -> (city: String, confidence: Float)? {
        guard let pixelBuffer = image.pixelBuffer(width: Int(inputSize), height: Int(inputSize)) else {
            return nil
        }
        
        do {
            try interpreter?.copy(pixelBuffer, toInputAt: 0)
            try interpreter?.invoke()
            
            let outputTensor = try interpreter?.output(at: 0)
            guard let outputData = outputTensor?.data as? Data else {
                return nil
            }
            
            let predictions = [Float](unsafeBytes: outputData)
            if let maxIndex = predictions.indices.max(by: { predictions[$0] < predictions[$1] }) {
                return (cityNames[maxIndex], predictions[maxIndex])
            }
        } catch {
            print("Inference error: \\(error)")
        }
        
        return nil
    }
    
    func getTopPredictions(from image: UIImage, k: Int = 5) -> [(city: String, confidence: Float)]? {
        guard let pixelBuffer = image.pixelBuffer(width: Int(inputSize), height: Int(inputSize)) else {
            return nil
        }
        
        do {
            try interpreter?.copy(pixelBuffer, toInputAt: 0)
            try interpreter?.invoke()
            
            let outputTensor = try interpreter?.output(at: 0)
            guard let outputData = outputTensor?.data as? Data else {
                return nil
            }
            
            let predictions = [Float](unsafeBytes: outputData)
            let predictions_with_cities = cityNames.enumerated().map { (idx, city) in
                (city: city, confidence: predictions[idx])
            }
            
            return Array(predictions_with_cities.sorted { $0.confidence > $1.confidence }.prefix(k))
        } catch {
            print("Inference error: \\(error)")
        }
        
        return nil
    }
}

// Extension to convert UIImage to CVPixelBuffer
extension UIImage {
    func pixelBuffer(width: Int, height: Int) -> CVPixelBuffer? {
        let attrs = [kCVPixelBufferCGImageCompatibilityKey: kCFBooleanTrue,
                     kCVPixelBufferCGBitmapContextCompatibilityKey: kCFBooleanTrue] as CFDictionary
        var pixelBuffer: CVPixelBuffer?
        let status = CVPixelBufferCreate(kCFAllocatorDefault,
                                        width,
                                        height,
                                        kCVPixelFormatType_32ARGB,
                                        attrs,
                                        &pixelBuffer)
        
        guard status == kCVReturnSuccess, let pixelBuffer = pixelBuffer else {
            return nil
        }
        
        CVPixelBufferLockBaseAddress(pixelBuffer, CVPixelBufferLockFlags(rawValue: 0))
        guard let context = CGContext(data: CVPixelBufferGetBaseAddress(pixelBuffer),
                                     width: width,
                                     height: height,
                                     bitsPerComponent: 8,
                                     bytesPerRow: CVPixelBufferGetBytesPerRow(pixelBuffer),
                                     space: CGColorSpaceCreateDeviceRGB(),
                                     bitmapInfo: CGImageAlphaInfo.noneSkipFirst.rawValue) else {
            return nil
        }
        
        UIGraphicsPushContext(context)
        draw(in: CGRect(x: 0, y: 0, width: width, height: height))
        UIGraphicsPopContext()
        
        CVPixelBufferUnlockBaseAddress(pixelBuffer, CVPixelBufferLockFlags(rawValue: 0))
        
        return pixelBuffer
    }
}

// Usage
let cityLocator = CityLocatorTFLite()
if let result = cityLocator?.predictCity(from: image) {
    print("Predicted: \\(result.city) (\\(result.confidence * 100)%)")
}

if let topPredictions = cityLocator?.getTopPredictions(from: image, k: 5) {
    for (city, confidence) in topPredictions {
        print("\\(city): \\(Int(confidence * 100))%")
    }
}
"""

print("iOS Swift Implementation for TFLite Model")
print("=" * 70)
print(ios_code)

### iOS Swift Example

In [ ]:
# Android Kotlin code for TFLite inference
android_code = """
import org.tensorflow.lite.Interpreter
import org.tensorflow.lite.gpu.CompatibilityList
import org.tensorflow.lite.support.image.TensorImage
import org.tensorflow.lite.support.image.ImageProcessor
import org.tensorflow.lite.support.image.ops.ResizeOp
import org.tensorflow.lite.support.common.ops.NormalizeOp
import org.tensorflow.lite.support.common.TensorProcessor
import android.graphics.Bitmap
import android.content.Context
import java.io.FileInputStream
import java.nio.MappedByteBuffer
import java.nio.channels.FileChannel

class CityLocatorTFLite(context: Context) {
    private lateinit var interpreter: Interpreter
    private val modelFileName = "city_classifier_quantized.tflite"
    private val INPUT_SIZE = 224
    private val NUM_CLASSES = 23
    
    private val cityNames = listOf(
        "Bangkok", "Barcelona", "Boston", "Brussels", "BuenosAires",
        "Chicago", "Lisbon", "London", "LosAngeles", "Madrid",
        "Medellin", "Melbourne", "MexicoCity", "Miami", "Minneapolis",
        "OSL", "Osaka", "PRG", "PRS", "Phoenix", "Rome", "TRT", "WashingtonDC"
    )
    
    init {
        val model = loadModelFile(context, modelFileName)
        val options = Interpreter.Options()
        
        // Enable GPU acceleration if available
        if (CompatibilityList().isDelegateSupportedOnThisDevice) {
            options.addDelegate(GpuDelegate())
        } else {
            options.setNumThreads(4)
        }
        
        interpreter = Interpreter(model, options)
    }
    
    fun predictCity(bitmap: Bitmap): Pair<String, Float> {
        // Prepare image
        var tensorImage = TensorImage(org.tensorflow.lite.DataType.FLOAT32)
        tensorImage.load(bitmap)
        
        // Process image: resize to 224x224 and normalize
        val imageProcessor = ImageProcessor.Builder()
            .add(ResizeOp(INPUT_SIZE, INPUT_SIZE, ResizeOp.ResizeMethod.BILINEAR))
            .add(NormalizeOp(127.5f, 127.5f))  // Normalize to [-1, 1]
            .build()
        
        tensorImage = imageProcessor.process(tensorImage)
        
        // Run inference
        val output = Array(1) { FloatArray(NUM_CLASSES) }
        interpreter.run(tensorImage.buffer, output)
        
        // Get prediction
        val predictions = output[0]
        val maxIndex = predictions.indices.maxByOrNull { predictions[it] } ?: 0
        val confidence = predictions[maxIndex]
        
        return Pair(cityNames[maxIndex], confidence)
    }
    
    fun getTopPredictions(bitmap: Bitmap, k: Int = 5): List<Pair<String, Float>> {
        var tensorImage = TensorImage(org.tensorflow.lite.DataType.FLOAT32)
        tensorImage.load(bitmap)
        
        val imageProcessor = ImageProcessor.Builder()
            .add(ResizeOp(INPUT_SIZE, INPUT_SIZE, ResizeOp.ResizeMethod.BILINEAR))
            .add(NormalizeOp(127.5f, 127.5f))
            .build()
        
        tensorImage = imageProcessor.process(tensorImage)
        
        val output = Array(1) { FloatArray(NUM_CLASSES) }
        interpreter.run(tensorImage.buffer, output)
        
        val predictions = output[0]
        return predictions.mapIndexed { idx, conf -> Pair(cityNames[idx], conf) }
            .sortedByDescending { it.second }
            .take(k)
    }
    
    private fun loadModelFile(context: Context, modelName: String): MappedByteBuffer {
        val assetFileDescriptor = context.assets.openFd(modelName)
        val fileInputStream = FileInputStream(assetFileDescriptor.fileDescriptor)
        val fileChannel = fileInputStream.channel
        val startOffset = assetFileDescriptor.startOffset
        val declaredLength = assetFileDescriptor.declaredLength
        return fileChannel.map(FileChannel.MapMode.READ_ONLY, startOffset, declaredLength)
    }
    
    fun close() {
        interpreter.close()
    }
}

// Usage
val cityLocator = CityLocatorTFLite(context)
val (predictedCity, confidence) = cityLocator.predictCity(bitmap)
val topPredictions = cityLocator.getTopPredictions(bitmap, k=5)

for ((city, conf) in topPredictions) {
    println("$city: ${(conf * 100).toInt()}%")
}

cityLocator.close()
"""

print("Android Kotlin Implementation for TFLite Model")
print("=" * 70)
print(android_code)

### Android Kotlin Example

In [ ]:
# Load and test the TFLite model
print("Testing TFLite model inference...\n")

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='city_classifier_quantized.tflite')
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("TFLite Model Details:")
print(f"  Input shape: {input_details[0]['shape']}")
print(f"  Input type: {input_details[0]['dtype']}")
print(f"  Output shape: {output_details[0]['shape']}")
print(f"  Output type: {output_details[0]['dtype']}\n")

# Test on a sample image from test dataset
print("Running inference on test image...")
for test_images, test_labels in test_dataset.take(1):
    sample_image = test_images[0:1]
    true_label_idx = np.argmax(test_labels[0])
    
    # Prepare input
    test_image = sample_image.numpy().astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    
    # Get output
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_idx = np.argmax(output_data[0])
    confidence = output_data[0][predicted_idx]
    
    print(f"True city:      {class_names[true_label_idx]}")
    print(f"Predicted city: {class_names[predicted_idx]}")
    print(f"Confidence:     {confidence*100:.2f}%")
    print(f"Correct:        {'✓ YES' if true_label_idx == predicted_idx else '✗ NO'}")
    
    print(f"\nTop 5 predictions:")
    top_5_idx = np.argsort(output_data[0])[-5:][::-1]
    for rank, idx in enumerate(top_5_idx, 1):
        print(f"  {rank}. {class_names[idx]:20s} - {output_data[0][idx]*100:6.2f}%")

print("\n✓ TFLite model working correctly!")


## Part 10: Mobile Deployment Code Examples

Use the generated TFLite model in your mobile apps with these code snippets.

In [ ]:
# Execute the full mobile optimization pipeline
print("Starting mobile optimization pipeline...")
print("This may take 5-10 minutes...\n")

pruned_path, tflite_path = apply_pruning_quantization_tflite(
    model,
    target_sparsity=0.50,
    fine_tune_epochs=2
)

print(f"\n✓ Mobile optimization complete!")
print(f"\nReady to deploy on mobile devices:")
print(f"  • Android: Use TensorFlow Lite Runtime")
print(f"  • iOS: Use Core ML or TFLite interpreter")
print(f"\nModel files generated:")
print(f"  • {tflite_path}")
print(f"  • {pruned_path}")
